In [ ]:
!pip show roboflow ultralytics


In [ ]:
from roboflow import Roboflow
import os
from ultralytics import YOLO
from datetime import datetime
import cv2
from pathlib import Path
import shutil
from glob import glob

## Descarregar el dataset

In [ ]:
rf = Roboflow(api_key="5HYzrdPm6LO8xmXDVn3G")
project = rf.workspace("vc").project("matriculas-espanolas")
version = project.version(1)
dataset = version.download("yolov11")                

## Entrenar 

In [ ]:
model = YOLO("yolo11n.pt")
results = model.train(data=os.path.join(dataset.location, "data.yaml"), epochs=50, imgsz=640)

## Entrenar el model

In [ ]:
model = YOLO("yolo11n.pt")
results = model.train(data=os.path.join(dataset.location, "data.yaml"), epochs=50, imgsz=640)

## Guardar el model

In [ ]:
models_path = "models"
fecha_hora_actual = datetime.now().strftime("%d_%m_%Y_%H_%M")
model_name = f"model_{fecha_hora_actual}"
model_path = os.path.join(models_path, model_name)
os.makedirs(models_path, exist_ok=True)
model.save(f"{model_path}.pt")

## Carregar el model

In [ ]:
model_matr = YOLO("models/model_27_03_2025_19_30.pt")

## Prediure imagues

In [ ]:
import os
from glob import glob  # Importamos la función glob directamente

def get_image_paths(input_path):
    """Obtiene la lista de rutas de imágenes a procesar"""
    if isinstance(input_path, list):
        return input_path
    elif os.path.isfile(input_path):
        return [input_path]
    elif os.path.isdir(input_path):
        image_paths = (glob(os.path.join(input_path, '*.jpg')) + 
                      glob(os.path.join(input_path, '*.png')) + 
                      glob(os.path.join(input_path, '*.jpeg')))
        return image_paths
    else:
        raise ValueError("Input path no es válido (debe ser imagen, lista o carpeta)")

def generar_markdown_imagenes(input_dir, output_md_path):
    """Genera un archivo Markdown con todas las imágenes de un directorio"""
    if not os.path.exists(input_dir):
        raise FileNotFoundError(f"El directorio {input_dir} no existe")
    
    imagenes = sorted(glob(os.path.join(input_dir, '*.jpg')) + 
                    glob(os.path.join(input_dir, '*.png')) + 
                    glob(os.path.join(input_dir, '*.jpeg')))
    
    contenido_md = "# Imágenes generadas\n\n"
    for img_path in imagenes:
        img_name = os.path.basename(img_path)
        ruta_relativa = os.path.relpath(img_path, start=os.path.dirname(output_md_path))
        contenido_md += f"## {img_name}\n\n"
        contenido_md += f"![{img_name}]({ruta_relativa})\n\n"
    
    os.makedirs(os.path.dirname(output_md_path), exist_ok=True)
    
    with open(output_md_path, 'w') as md_file:
        md_file.write(contenido_md)
    
    return output_md_path
    

In [ ]:
input_path = "Matriculas-Españolas-1/valid/images/"
image_paths = get_image_paths(input_path)

name = "images"
project = "./output_model"
results = model_matr.predict(
    source=image_paths,
    conf=0.25,
    save=True,
    save_txt=True,
    name = name,
    project = project,
    exist_ok=True,
    verbose=False
)

origen = os.path.join(project, name, "labels")
destino = os.path.join(project, "labels")

shutil.copytree(origen, destino, dirs_exist_ok=True)
shutil.rmtree(origen)

md_path = os.path.join(project, "imagenes_generadas.md")
image_paths = os.path.join(project, name)

generar_markdown_imagenes(image_paths, md_path)

Results saved to output_model/images
6 labels saved to output_model/images/labels


'./output_model/imagenes_generadas.md'

## Retallar matricules

In [34]:
def predecir_yolo(photo, model):
    results = model.predict(
        source=photo,
        conf=0.25,
        save=False,
        save_txt=False,
        verbose=False
    )
    return results[0]

def obtener_bbox(result, max_len=1):
    boxes_with_confs = [(box.xyxy, box.conf.item()) for box in result.boxes]
    boxes_sorted = sorted(boxes_with_confs, key=lambda x: x[1], reverse=True)
    recuadros = [box[0] for box in boxes_sorted[:max_len]]
    return recuadros

def recortar_y_guardar_matricula(bboxes, photo, output_dir="recortes_matriculas"):
    """Recorta las matriculas y las guarda en una carpeta"""
    img = cv2.imread(photo)
    if img is None:
        raise ValueError(f"No se pudo leer la imagen: {photo}")
    
    recortes = []
    for i, bbox in enumerate(bboxes):
        coords = bbox.tolist()[0]
        x1, y1, x2, y2 = map(int, coords)
        
        h, w = img.shape[:2]
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)
        
        if x2 > x1 and y2 > y1:
            recorte = img[y1:y2, x1:x2]
            recortes.append(recorte)
            
            os.makedirs(output_dir, exist_ok=True)
            ruta_guardado = os.path.join(output_dir, f"{os.path.basename(photo)}.jpg")
            cv2.imwrite(ruta_guardado, recorte)
            print(f"Guardado: {ruta_guardado}")
        else:
            print(f"Advertencia: Bbox inválida {[x1, y1, x2, y2]}")
    
    return recortes

def obtener_matriculas(photo, model, max_len=1):
    result = predecir_yolo(photo, model)
    bboxes = obtener_bbox(result, max_len)
    recortes = recortar_y_guardar_matricula(bboxes, photo)
    return recortes

In [35]:
img = "./Matriculas-Españolas-1/train/images"
output_folder = "./matriculas_recortadas"

imges_path = get_image_paths(img)
for img_path in imges_path:    
    obtener_matriculas(img_path, model_matr)

Guardado: recortes_matriculas/1668451260013_jpg.rf.a43de5c4c61524722133953d593db167.jpg.jpg
Guardado: recortes_matriculas/1668451259934_jpg.rf.3928be91786172e8a9d60b940da09170.jpg.jpg
Guardado: recortes_matriculas/1668451259821_jpg.rf.741a09f814557ec2d6e847374049b634.jpg.jpg
Guardado: recortes_matriculas/1668451259976_jpg.rf.7b05048043432430bd0cb020d8d0ad07.jpg.jpg
Guardado: recortes_matriculas/1668451259976_jpg.rf.8e1a4c9dd4297c481a772867815068b5.jpg.jpg
Guardado: recortes_matriculas/1668448029142_jpg.rf.cf6da3a11178b60f2fa19e86ccf62eea.jpg.jpg
Guardado: recortes_matriculas/1668451259895_jpg.rf.62f5495482fc6cf6445129c672f08271.jpg.jpg
Guardado: recortes_matriculas/1668451259760_jpg.rf.053d2c6ac7893848063fc1751bcac745.jpg.jpg
Guardado: recortes_matriculas/1668448029123_jpg.rf.88cffbc2f7981d8606a5f51f15ccee46.jpg.jpg
Guardado: recortes_matriculas/1668451259821_jpg.rf.b88a30365cd7a403da97337f767a853f.jpg.jpg
Guardado: recortes_matriculas/1668451259915_jpg.rf.117ccdff1a48ba855cab3b94d0dec